In [1]:

# WARNING - disk size of data file is about 5.4GB, and after reading it into RAM
# R process uses about 6.4GB of memory.
DATA_FNAME = "MINI_TRIMMED.csv" # "clash_royale_games_Jan_2019.csv"
library(data.table)
data = fread(DATA_FNAME)
DATA_LEN = length(data$timestamp)




Warning message in fread(DATA_FNAME):
“Detected 3 column names but the data has 9 columns (i.e. invalid file). Added 6 extra default column names at the end.”


In [2]:
ordered = data[order(data$timestamp)]
head(ordered, n=10)

timestamp,player,deck,V4,V5,V6,V7,V8,V9
<dttm>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2018-12-31 23:00:46,elixir-collector,golem,mega-knight,mega-minion,night-witch,rage,witch,zap
2018-12-31 23:01:34,dark-prince,goblin-gang,miner,pekka,the-log,tornado,wizard,zap
2018-12-31 23:02:00,baby-dragon,balloon,barbarian-barrel,bowler,freeze,inferno-dragon,lumberjack,tornado
2018-12-31 23:02:00,baby-dragon,fireball,inferno-dragon,mega-minion,miner,royal-giant,the-log,tombstone
2018-12-31 23:02:02,archers,barbarian-barrel,goblins,hog-rider,knight,poison,tesla,zap
2018-12-31 23:02:21,goblin-barrel,hog-rider,inferno-tower,minion-horde,prince,rocket,skeleton-army,wizard
2018-12-31 23:03:12,arrows,balloon,electro-wizard,ice-wizard,inferno-dragon,mega-knight,minions,tombstone
2018-12-31 23:03:42,baby-dragon,balloon,barbarian-barrel,bowler,freeze,inferno-dragon,lumberjack,tornado
2018-12-31 23:04:20,dart-goblin,electro-wizard,giant-skeleton,goblin-barrel,lumberjack,the-log,valkyrie,zap


In [44]:
library(arules);

# Python-like fstrings.
library(glue);

In [4]:
transactions = arules::read.transactions(DATA_FNAME, format="basket", head=1, sep=',', cols=1)

In [5]:
inspect(head(transactions))

    items                     transactionID
[1] {elixir-collector,                     
     golem,                                
     mega-knight,                          
     mega-minion,                          
     night-witch,                          
     rage,                                 
     witch,                                
     zap}              2018-12-31T23:00:46Z
[2] {dark-prince,                          
     goblin-gang,                          
     miner,                                
     pekka,                                
     the-log,                              
     tornado,                              
     wizard,                               
     zap}              2018-12-31T23:01:34Z
[3] {baby-dragon,                          
     balloon,                              
     barbarian-barrel,                     
     bowler,                               
     freeze,                               
     inferno-dragon,            

In [34]:
minSupport = 0.01
frequentItemSets = apriori(transactions, 
                           parameter = list(supp = minSupport, 
                                            minlen = 1, 
                                            target = "frequent itemsets"),
                           appearance = NULL, control = list(verbose = FALSE))

# frequentItemSets = eclat(transactions, parameter = list(supp = minSupport, 
#                                                         minlen = 2, 
#                                                         target = "frequent itemsets",
#                                                         tidLists = FALSE),
#                               control = list(verbose = FALSE))

In [96]:

n_sets = function(n) arules::sort(frequentItemSets[which(size(frequentItemSets) == n)], decreasing=TRUE)
m = function(x) methods(class=class(x))
p = function(x) print(glue(x))

In [55]:

inspect(head(sort(n_sets(8), decreasing=TRUE)))


    items        support transIdenticalToItemsets count
[1] {cannon,                                           
     fireball,                                         
     hog-rider,                                        
     ice-golem,                                        
     ice-spirit,                                       
     musketeer,                                        
     skeletons,                                        
     the-log}     0.0158                   0.0158   158


In [98]:
a = n_sets(1)
inspect(head(a))
l = labels(a[1])# [1]
s = support(a[1], transactions)
p("Here is the list of most popular cards (overall). The most popular card, the {l} was used in {100 * s}% of all decks!")


    items         support transIdenticalToItemsets count
[1] {zap}         0.4319  0                        4319 
[2] {fireball}    0.3424  0                        3424 
[3] {the-log}     0.3255  0                        3255 
[4] {hog-rider}   0.2816  0                        2816 
[5] {goblin-gang} 0.2715  0                        2715 
[6] {wizard}      0.2269  0                        2269 
Here is the list of most popular cards (overall). The most popular card, the {zap} was used in 43.19% of all decks!
